<a href="https://colab.research.google.com/github/sahareslami/Feature_Selection/blob/master/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Selection
writer : sahar shekholeslami

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import time
import pickle
import random

# Downlaod and Save data
First, we save all the features of the training data and their labels in one file. and do the same for test data.




In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1IhZsc4bRIbnLtAEwAxhHBbaj385_n1ET

Downloading...
From: https://drive.google.com/u/0/uc?id=1IhZsc4bRIbnLtAEwAxhHBbaj385_n1ET
To: /content/FinalDataset.zip
9.88MB [00:00, 87.1MB/s]


simply unizp the file!

In [ ]:
!unzip "/content/FinalDataset.zip"

The saved format is csv, which is why we use dataframe from pandas for store data.


In [ ]:
totalTrainX = pd.read_csv('/content/train/features.csv' , header=None)
totalTrainY = pd.read_csv( '/content/train/Labels.csv' , header=None)
testX = pd.read_csv('/content/test/features.csv' , header=None)
testY = pd.read_csv('/content/test/Labels.csv' , header=None)


top_fiveTrainX contains first five data from each category that we want to train our classification algorithm, with in the following


In [ ]:
top_fiveTrainX = pd.DataFrame()
top_fiveTrainY = pd.DataFrame()
cntOFData = np.zeros((16), dtype=int)
for (index , label) in totalTrainY.iterrows():
  if cntOFData[label[0]] < 5:
    top_fiveTrainX = top_fiveTrainX.append(totalTrainX.iloc[index])
    top_fiveTrainY = top_fiveTrainY.append(totalTrainY.iloc[index])
    cntOFData[label[0]] += 1

Similarly, top_TenTrainX contains first ten data from each category

In [ ]:
top_TenTrainX = pd.DataFrame()
top_TenTrainY = pd.DataFrame()
cntOFData2 = np.zeros((16), dtype=int)

for (index , label) in totalTrainY.iterrows():
  if cntOFData2[label[0]] < 10:
    top_TenTrainX = top_TenTrainX.append(totalTrainX.iloc[index])
    top_TenTrainY = top_TenTrainY.append(totalTrainY.iloc[index])
    cntOFData2[label[0]] += 1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Random Forests

In random forests , each tree in the ensemble is built from a sample drawn with replacement (i.e., a bootstrap sample) from the training set.

Furthermore, when splitting each node during the construction of a tree, the best split is found either from all input features or a random subset of size max_features. (See the parameter tuning guidelines for more details).

The purpose of these two sources of randomness is to decrease the variance of the forest estimator. Indeed, individual decision trees typically exhibit high variance and tend to overfit. The injected randomness in forests yield decision trees with somewhat decoupled prediction errors. By taking an average of those predictions, some errors can cancel out. Random forests achieve a reduced variance by combining diverse trees, sometimes at the cost of a slight increase in bias. In practice the variance reduction is often significant hence yielding an overall better model.

In contrast to the original publication [B2001], the scikit-learn implementation combines classifiers by averaging their probabilistic prediction, instead of letting each classifier vote for a single class.
[Breiman, “Random Forests”, Machine Learning](https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf)


In [ ]:
def randomForestClassifier(tx , ty , vx , vy):
  clf = RandomForestClassifier(n_estimators= 500 , max_depth=30, random_state=0 , criterion = "gini")
  clf.fit(tx, ty)
  predY = clf.predict(vx)
  return metrics.accuracy_score(vy, predY)

**About Random Forest in Sci_ki learn and Parameters:  \\
n_estimatorsint:** \\
The number of trees in the forest. \\
**criterion :** \\
The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. \\
**max_depth:** \\
The maximum depth of the tree \\
**max_features:**  
The number of features to consider when looking for the best split 

1.   If int, then consider max_features features at each split.
2.   If float, then max_features is a fraction and int(max_features * n_features) features are considered at each split
3. If “auto”, then max_features=sqrt(n_features).

4. If “sqrt”, then max_features=sqrt(n_features) (same as “auto”).

5. If “log2”, then max_features=log2(n_features).

6. If None, then max_features=n_features.

**Results**

n_estimators | max_depth | criterion | accuracy
--- | --- | --- | --- 
100 |10 | entropy | 0.9642857142857143
100 | 30| entropy | 0.9649859943977591 
500 | 30 | entropy  | 0.9744397759103641
500 | 30 | gini  | 0.979890756302521
800 | 30 | entropy | 0.9744397759103641
800 | 30 | gini | 0.9793417366946778 

The best accuracy is achieved when n_estimatorsint = 500 , max_depth = 30 and criterion = "gini"

In [ ]:
# for setting Hyper parameters
trainX = totalTrainX.head(250)
trainY = totalTrainY.head(250)
validationX = totalTrainX.tail(100)
validationY = totalTrainY.tail(100)
randomForestClassifier(trainX , trainY , validationX , validationY)

# Multi-layer Perceptron 
**Multi-layer Perceptron (MLP)** is a supervised learning algorithm that learns a function $f(\cdot): R^m \rightarrow R^o$ by training on a dataset, where  is the number of dimensions for input and  is the number of dimensions for output. Given a set of features $X = {x_1, x_2, ..., x_m}$
nd a target , it can learn a non-linear function approximator for either classification or regression. It is different from logistic regression, in that between the input and the output layer, there can be one or more non-linear layers, called hidden layers. Figure 1 shows a one hidden layer MLP with scalar output.

![MLP](https://doc-0k-5s-docs.googleusercontent.com/docs/securesc/kc1q400hlmlt1l5c0e2qjjelvmpo2scd/rntaop9j60pmlomiqs320j1lj12dq0k2/1594899675000/14271364715593122408/14271364715593122408/1AddtblRmBfSTu0WZeckmsLCf3FSlqEdv?e=download&authuser=0)


In [ ]:
def NNClassifier(tx, ty , vx , vy):
  clf = MLPClassifier(random_state=1,hidden_layer_sizes = (100,100) ,  activation = "logistic"  , max_iter=500)
  clf.fit(tx, ty)
  predY = clf.predict(vx)
  return metrics.accuracy_score(vy, predY)

**About MLP Classifier in Sci_ki learn and Parameters:** \\
This model optimizes the log-loss function using LBFGS or stochastic gradient descent. \\

** hidden_layer_sizes ** 
The ith element represents the number of neurons in the ith hidden layer. 
** activation : ** \\
Activation function for the hidden layer \\
1. ‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x \\
2. ‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
3. ‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
4. relu’, the rectified linear unit function, returns f(x) = max(0, x)

**learning_rate** \\
Learning rate schedule for weight updates.

**Results**

hidden_layer_sizes | activation  | accuracy
--- | --- | --- 
(100,100) | relu | 0.91  
(100,100) | tanh | 0.95 
(100,100) | logistic | 0.96
(200,200) | tanh | 0.94
(200,200) | relu | 0.94
(200,200) | logistic  | 0.96

The best accuracy is achieved when hidden_layer_sizes have 100 neurons both and activation function is logistic.

In [ ]:
# for setting Hyper parameters
trainX = totalTrainX.head(250)
trainY = totalTrainY.head(250)
validationX = totalTrainX.tail(100)
validationY = totalTrainY.tail(100)
NNClassifier(trainX , trainY , validationX , validationY)

# Support Vector Machine
The objective of the support vector machine algorithm is to find a hyperplane in an N-dimensional space(N — the number of features) that distinctly classifies the data points.

In [ ]:
def SVMClassifier(tx , ty , vx , vy):
  clf = make_pipeline(StandardScaler(), SVC(gamma='auto' , kernel = 'sigmoid'))
  clf.fit(tx , ty)
  predY = clf.predict(vx)
  return metrics.accuracy_score(vy, predY)

**About SVM in Sci_kit learn and Parameters: \
kernel :**

Specifies the kernel type to be used in the algorithm. It must be one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ or a callable. If none is given, ‘rbf’ will be used. If a callable is given it is used to pre-compute the kernel matrix from data matrices.

**Results**

kernel | accuracy
--- | ---  
linear | 0.95  
poly | 0.1 :(
rbf | 0.89
sigmoid | 0.94

The data are almost linearly separable so using diffrent kerenls doesnot effective!

In [ ]:
# for setting Hyper parameters
trainX = totalTrainX.head(250)
trainY = totalTrainY.head(250)
validationX = totalTrainX.tail(100)
validationY = totalTrainY.tail(100)
print(SVMClassifier(trainX , trainY , validationX , validationY))


# Decision Trees
Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.

In [ ]:
def decisionTreeClassifier(tx , ty , vx , vy):
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(tx, ty)
  predY = clf.predict(vx)
  return metrics.accuracy_score(vy, predY)

This function determines the accuracy of the input features on the train and validation data based on the input's classifier 

In [ ]:
def find_performance(features , labels , approach):
  trainX = features.head(250)
  trainY = labels.head(250)
  validationX = features.tail(100)
  validationY = labels.tail(100)
  if approach == 'svm':
    return SVMClassifier(trainX ,trainY, validationX, validationY)
  if approach == 'DT':
    return decisionTreeClassifier(trainX ,trainY, validationX, validationY)
    

# Forward Feature Selection

In this approach, each time we select a feature that is have higher accuracy with the selected features.
In the method below,add one feature to selected features in a way that maximizes accuracy

In [ ]:
def add_One_Feature(selectedFeatures , otherFeatures , selectedIndex):
  max_acc , max_idx = 0 , 0
  nxt = len(select.columns)
  
  for (index , feature) in otherFeatures.iteritems():
    selectedFeatures.insert(nxt ,nxt ,feature)
    cur_acc = find_performance(selectedFeatures ,totalTrainY , 'DT')
    if cur_acc > max_acc:
      max_acc = cur_acc
      max_idx = index
    selectedFeatures.drop(nxt , axis=1, inplace=True) 

  selectedFeatures.insert(nxt ,nxt ,other[max_idx])
  otherFeatures = otherFeatures.drop(max_idx , axis=1)
  selectedIndex = np.append(selectedIndex , [max_idx])

  return selectedFeatures,otherFeatures,selectedIndex

In [ ]:
def forwardFeatureSelection(select,other,indexs, numberOfFeature):
  start_time = time.time()

  for i in range(0,numberOfFeature):
    select,other,indexs = add_One_Feature(select ,other ,indexs)
    
  result = pd.DataFrame(indexs)
  result.to_csv(r"/content/forwardFeatureSelection.csv" ,  index = False, header=True)

  with open('/content/forwardFeatureSelection.csv' , 'wb') as handle:
    pickle.dump(indexs, handle)
  print("time" , time.time() - start_time)
  return indexs


# Backward Feature Selection
In this method, each time we remove a feature that removal of that feature from the set of features has less effect on reducing accuracy.
The following two functions remove a feature in such a way that it has maximum accuracy.

In [ ]:
def reduced_feature(select,idx):
  select = select.drop(columns = [idx])
  return find_performance(select , totalTrainY , "DT")

In [ ]:
def delete_one_feature(selectedFeatures , selectedIndex):
  max_acc , max_idx = 0 , 0
  for (index , feature) in selectedFeatures.iteritems():
      cur_acc = reduced_feature(selectedFeatures , index)
      if cur_acc > max_acc:
        max_acc = cur_acc
        max_idx = index
  selectedFeatures = selectedFeatures.drop(columns=[max_idx])
  selectedIndex = np.append(selectedIndex , [max_idx])
  return selectedFeatures , selectedIndex

In [ ]:
def preprocess(trainData):
  max_acc , idx = 0 , 0
  for (index,feature) in trainData.iteritems():
    acc = performanceBaseOnDT(feature.values.reshape(-1,1) , trainY)
    if acc > max_acc:
      idx = index
      max_acc = acc
    # print(index , acc)
    if acc <= 0.2:
      trainData = trainData.drop(columns = [index])
  print("iki" , trainData[idx])
  return trainData    

In [ ]:
def backwardFeatureSelection_v1(features , indexs , numberOfFeature):
  start_time = time.time()
  
  features_collection = {} 
  indexs_collection = {}


  for i in range(0,8):
    st_index , en_index = i * 256 , (i + 1) * 256
    features_collection[i] = pd.DataFrame(features.iloc[:,st_index :  en_index])
    indexs_collection[i] =  np.array([], dtype = np.int)


  fraction = int((4096 - numberOfFeature) / 8)

  for i in range(0 , 8):
      for j in range(0 , fraction):
        print(j)
        features_collection[i] , indexs_collection[i] = delete_one_feature(features_collection[i] ,indexs_collection[i])
      indexs = np.append(indexs, indexs_collection[i])

  result = pd.DataFrame(indexs)
  result.to_csv(r"/content/backwardFeatureSelectionv2.csv" ,  index = False, header=True)

  with open('/content/backwardFeatureSelectionv2.pickle') as handle:
    pickle.dump(indexs, handle)
    print(time.time() - start_time)
  return features_collection[0],indexs

# Ada Boost
The core principle of AdaBoost is to fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction. The data modifications at each so-called boosting iteration consist of applying weights $w_1,w_2, ..., w_n$ to each of the training samples. Initially, those weights are all set to $w_i = 1/N$ , so that the first step simply trains a weak learner on the original data. For each successive iteration, the sample weights are individually modified and the learning algorithm is reapplied to the reweighted data. At a given step, those training examples that were incorrectly predicted by the boosted model induced at the previous step have their weights increased, whereas the weights are decreased for those that were predicted correctly. As iterations proceed, examples that are difficult to predict receive ever-increasing influence. Each subsequent weak learner is thereby forced to concentrate on the examples that are missed by the previous ones in the sequence


In [ ]:
def adaBoost(tx,ty,vx,vy):
  start_time = time.time()
  clf = AdaBoostClassifier(
    tree.DecisionTreeClassifier(max_depth=1),
    n_estimators = 800 , learning_rate = 0.1)
  clf.fit(tx, ty)
  predY = clf.predict(vx)

  important_features_dict = {}
  for x,i in enumerate(clf.feature_importances_):
    important_features_dict[x]=i

  important_features_lt = sorted(important_features_dict.items(),key=lambda x: x[1], reverse=True)

  indexs = np.array([], dtype = np.int)
  for x,i in important_features_lt:
    if i > 0.0:
      indexs = np.append(indexs, x)

  result = pd.DataFrame(indexs)
  result.to_csv(r"/content/adaboost.csv" ,  index = False, header=True)

  with open('adaboost.pickle', 'wb') as handle:
    pickle.dump(indexs, handle)

  return metrics.accuracy_score(vy, predY) 


**n_estimatorsint** \
The maximum number of estimators at which boosting is terminated. In case of perfect fit, the learning procedure is stopped early.

**learning_ratefloat** \
Learning rate shrinks the contribution of each classifier by learning_rate. There is a trade-off between learning_rate and n_estimators.

**Results**

n_estimatorsint | learning_ratefloat  | accuracy
--- | ---  | ---
500 | 0.1  | 0.77
500 | 0.2 |  0.73
800 | 0.1 |  0.79

In [ ]:

# for setting Hyper parameters
trainX = totalTrainX.head(250)
trainY = totalTrainY.head(250)
validationX = totalTrainX.tail(100)
validationY = totalTrainY.tail(100)
adaBoost(trainX , trainY , validationX , validationY)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.91

# Train 

In [ ]:
def train_with_selected_feature(indexs , approach):
  selected_feature_train_x = totalTrainX.iloc[:,indexs]
  selected_feature_test_x = testX.iloc[:,indexs]
  if approach == "RF":
    return randomForestClassifier(selected_feature_train_x , totalTrainY , selected_feature_test_x , testY)
  if approach == "SVM":
     return SVMClassifier(selected_feature_train_x , totalTrainY , selected_feature_test_x , testY) 
  if approach == "NN":
     return NNClassifier(selected_feature_train_x , totalTrainY , selected_feature_test_x , testY) 
  if approach == "DT":
     return decisionTreeClassifier(selected_feature_train_x , totalTrainY , selected_feature_test_x , testY)

In [ ]:
def train_with_five_data(indexs , approach):
  selected_feature_train_x = top_fiveTrainX.iloc[:,indexs]
  selected_feature_test_x = testX.iloc[:,indexs]
  if approach == "RF":
    return randomForestClassifier(selected_feature_train_x , top_fiveTrainY , selected_feature_test_x , testY)
  if approach == "SVM":
     return SVMClassifier(selected_feature_train_x , top_fiveTrainY , selected_feature_test_x , testY) 
  if approach == "NN":
     return NNClassifier(selected_feature_train_x , top_fiveTrainY , selected_feature_test_x , testY) 
  if approach == "DT":
     return decisionTreeClassifier(selected_feature_train_x , top_fiveTrainY , selected_feature_test_x , testY)

In [ ]:
def train_with_ten_data(indexs , approach):
  selected_feature_train_x = top_TenTrainX.iloc[:,indexs]
  selected_feature_test_x = testX.iloc[:,indexs]
  if approach == "RF":
    return 
  if approach == "SVM":
     return SVMClassifier(selected_feature_train_x ,  top_TenTrainY , selected_feature_test_x , testY) 
  if approach == "NN":
     return NNClassifier(selected_feature_train_x ,  top_TenTrainY , selected_feature_test_x , testY) 
  if approach == "DT":
     return decisionTreeClassifier(selected_feature_train_x ,  top_TenTrainY , selected_feature_test_x , testY)

# Results
Random Forrest algorithm is resistant to the number of data while it is highly vulnerable to the number of features.

number of data and the number of features affect the neural network algorithm

SVM is also affected by both

run the cell below for see the full results.

In [ ]:
final_result = pd.read_csv('/content/result.csv' , header=None)
final result

In [ ]:
classfierAlg = np.array(["SVM" , "NN" , "RF"])
num_feature = np.array([50 , 100 , 300])
all_feature = np.arange(4096)

with open('/content/forwardFeaturesSelection.pkl', 'rb') as handle:
    Findex = pickle.load(handle)

with open('/content/backwardFeatureSelectionv2.pickle', 'rb') as handle:
    Bindex = pickle.load(handle)

with open('/content/adaboost.pickle', 'rb') as handle:
    Aindex = pickle.load(handle)

result = pd.DataFrame(columns = ['classifier algorithm', 'number of training data' , 'feature selection algorithm' , 'numebr of feature' , 'accuracy' ])

for alg in classfierAlg:
  for num in num_feature:
    a = train_with_selected_feature(Findex[0:num],alg)
    b = train_with_ten_data(Findex[0:num],alg)
    c = train_with_five_data(Findex[0:num],alg)
    result = result.append({'classifier algorithm' : alg  , 'number of training data' : 5, 'feature selection algorithm' : "forward" , 'numebr of feature' : num , 'accuracy' : c} , ignore_index=True)
    result = result.append({'classifier algorithm' : alg  , 'number of training data' : 10, 'feature selection algorithm' : 'forward' , 'numebr of feature' : num , 'accuracy' : b} , ignore_index=True)
    result = result.append({'classifier algorithm' : alg  , 'number of training data' : 30, 'feature selection algorithm' : "forward" , 'numebr of feature' : num , 'accuracy' : a} , ignore_index=True)



for alg in classfierAlg:
  a = train_with_selected_feature(Aindex,alg)
  b = train_with_ten_data(Aindex,alg)
  c = train_with_five_data(Aindex,alg)

  result = result.append({'classifier algorithm' : alg  , 'number of training data' : 5, 'feature selection algorithm' : "adaboost" , 'numebr of feature' : "-" , 'accuracy' : c} , ignore_index=True)
  result = result.append({'classifier algorithm' : alg  , 'number of training data' : 10, 'feature selection algorithm' : 'adaboost' , 'numebr of feature' : "-" , 'accuracy' : b} , ignore_index=True)
  result = result.append({'classifier algorithm' : alg  , 'number of training data' : 30, 'feature selection algorithm' : "adaboost" , 'numebr of feature' : "-" , 'accuracy' : a} , ignore_index=True)


bnum_feature = np.array([50 , 100 , 200])

for alg in classfierAlg:
  for num in bnum_feature:
    a = train_with_selected_feature(Bindex[0:num],alg)
    b = train_with_ten_data(Bindex[0:num],alg)
    c = train_with_five_data(Bindex[0:num],alg)
    result = result.append({'classifier algorithm' : alg  , 'number of training data' : 5, 'feature selection algorithm' : "backward" , 'numebr of feature' : num , 'accuracy' : c} , ignore_index=True)
    result = result.append({'classifier algorithm' : alg  , 'number of training data' : 10, 'feature selection algorithm' : 'backward' , 'numebr of feature' : num , 'accuracy' : b} , ignore_index=True)
    result = result.append({'classifier algorithm' : alg  , 'number of training data' : 30, 'feature selection algorithm' : "backward" , 'numebr of feature' : num , 'accuracy' : a} , ignore_index=True)


for alg in classfierAlg:
  a = train_with_selected_feature(all_feature,alg)
  b = train_with_ten_data(all_feature,alg)
  c = train_with_five_data(all_feature,alg)
  result = result.append({'classifier algorithm' : alg  , 'number of training data' : 5, 'feature selection algorithm' : "all_feature" , 'numebr of feature' : 4096 , 'accuracy' : c} , ignore_index=True)
  result = result.append({'classifier algorithm' : alg  , 'number of training data' : 10, 'feature selection algorithm' : 'all_feature' , 'numebr of feature' : 4096 , 'accuracy' : b} , ignore_index=True)
  result = result.append({'classifier algorithm' : alg  , 'number of training data' : 30, 'feature selection algorithm' : "all_feature" , 'numebr of feature' : 4096 , 'accuracy' : a} , ignore_index=True)


result.to_csv(r"/content/result.csv" ,  index = False, header=True)
with open('result.pickle', 'wb') as handle:
    pickle.dump(result, handle)